In [8]:
# Colocar porcentaje de ganancia y compra minima

porcentaje_ganancia = 7
compra_minima = 40000

print(f" \n \n Con una compra minima de ${compra_minima}, ganariamos ${(round(compra_minima / (1-(porcentaje_ganancia/100))) - compra_minima)}\n \n")

 
 
 Con una compra minima de $40000, ganariamos $3011
 



In [9]:
# definicion de palabras clave
palabras_clave = ['Body Advanced']

from os import nice
# Instalacion
!pip install tika

# Correccion
from tika import parser
raw = parser.from_file('Mayorista.pdf')
df = (raw['content'])
df = df.replace('\n',',')
df = df.replace(',,',',')
df = df.replace(',,,,,,,,,,,,,,,,,,,,','')

# Reemplazo de las comas en los precios (para luego dividir los productos por coma)
for i,v in enumerate(df):
  if i < len(df)-3:
    if v == ',':
      if df[i+2] == '0':
        df = df[:i] + '.' + df[i + 1:]

      try:
        valor1 = int(df[i+1])
        if type(valor1) == int:
          df = df[:i] + '.' + df[i + 1:]
      except:
        continue

      try:
        valor2 = int(df[i+2])
        if type(valor2) == int:
          df = df[:i] + '.' + df[i + 1:]
      except:
          continue

# Split
df = df.split(',')

# Agregar ganancia
porcentaje_ganancia = 1-(porcentaje_ganancia/100)
df1 = df.copy()
for i1,v1 in enumerate(df1):
  for i2,v2 in enumerate(v1):
    if v2 == '$':
      size = len(df1[i1])
      to_delete = len(df1[i1][i2+2:])
      mod_string = df1[i1][:size - to_delete]
      mod_string += str(round(int(df1[i1][i2+2:]) / porcentaje_ganancia))
      df1[i1] = mod_string

df = df1.copy()

# Quitar filas irrelevantes
for i,v in enumerate(df):
  if v.startswith('Fecha') or v.startswith('Página') or v.startswith('MARCA') or v.startswith('Lista'):
    df[i] = ''
  for palabra in palabras_clave:
    if palabra in v:
      valor = v
      break
    else:
      valor = ''
  df[i] = valor

# Eliminado de espacios vacios
while '' in df:
  df.remove('')

# Insertar marca
df1 = df.copy()
corrector = 0
for i,v in enumerate(df):

  # Obtener marca
  for i2,v2 in enumerate(v):
    try:
      int(v2)
      coordenada = i2
      break
    except:
      continue
  marca = v[0:coordenada-1]
  
  # Comparacion con la marca anterior
  try:
    if marca == df[i-1][0:len(marca)]:
      continue
    else:
      if i > 0:
        # Inserto marca
        df1.insert(i+corrector, '\n\n')
        #df1.insert(i+1+corrector, ' '*10 + marca + '\n')
        corrector = corrector + 1
        
      else:
        df1.insert(i+corrector, '\n')
        corrector = corrector + 1

        #df1.insert(i+corrector, ' '*10 + marca + '\n')
        #corrector = corrector + 1
        
  except:
    continue

df = df1.copy() 

# Encabezado piola
df.insert(0, f'ROYAL DISTRIBUIDORA \n\n  LISTA MAYORISTA CREATINAS \n\n COMPRA MINIMA: ${compra_minima} \n\n\n MARCA  -  CODIGO  -  PRODUCTO  -  PRECIO\n\n')
# Formato prolijo
n = '\n'.join(df)

# Guardado
text_file = open("Lista Mayorista de Creatinas.txt", "w")
text_file.write(n)
text_file.close()
from google.colab import files
files.download('Lista Mayorista de Creatinas.txt')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>